In [2]:
#!/usr/bin/env python3

import sys
import os
import glob
import itertools
import statistics

import pandas as pd
from collections import Counter, defaultdict, OrderedDict
from string import ascii_lowercase

from Bio import AlignIO, SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import MutableSeq, translate

from ind import findEnds
from output import printErrors

In [ ]:
# location of reference fasta files

In [49]:
def find_dna_subs(read, ref, verbose=False, start_offset=0, end_trail=0):
    """
    @ read, ref: MutableSeq objects
    :return errors - tuple (position, expected triplet, actual triplet, ) / none if broken read
    
    This is intended purely for counting PCR polymerase substitutions.
    The assumption is that the reference contains only the gene of interest.
    As for HGVS, the starting offset and number of trailing nt are variable
    Letter by letter report mutations in NGS read, all counts 1- based in result (code in 0-count for Python).
    - substitution: 78C = nt 78 in reference is changed to C, assuming reference starts with 1
    - deletions: 78d6 = 6 nt deleted starting with 78: 1-77, d6, 84-end
    - insertion: 78iATC = seq ATC inserted between 78 and 79
    """

    if read is None:
        if verbose:
            print('no read provided')
        return

#     # find the ends of the read
    ends = findEnds(read, ref, start_offset)

    # scan read & reference letter by letter, counting position in reference
    # reads have been trimmed so that reference starts @ offset=3 by default (0,1,2 is the extra triplet)
    dna_errors = []
    ref_index = ends.get('start') - start_offset
    i = ends.get('start')
    max_i = len(ref) - end_trail
    
    # check for indels
    if str(read).find('-') != -1:
        if verbose:
            print('indel')
        return
    elif str(ref).find('-') != -1:
        if verbose:
            print('indel')
        return
    elif len(ref) != len(read):
        if verbose:
            print('indel')
        return
    else:
        # continue on to scand the read for substitutions
        while i < ends.get('end'):
            if i > max_i:
                break
            # check for differences
            if read[i] == ref[i]:
                ref_index += 1
                i += 1

            else:
                # substitution
                # don't report N as mutations
                if str(read[i]) == "N" or str(read[i]) == "n":
                    i += 1
                    ref_index += 1
                    continue
                elif ref_index > 0:
                    dna_errors += [(str(ref_index + 1), 's', str(read[i]))]
                i += 1
                ref_index += 1


    return tuple(dna_errors)

In [12]:
os.chdir("../full_fasta")

The fixed length of sequences means that pre-aligning the sequences is not necessary. I can directly compare each FASTA read with the reference.

## Error rate of sequencing: the WT files

#### WT first half sequence

In [38]:
dna_variant_counts = {}
error_summary = {}

In [46]:
# start with the forward sequenced on the WT
with open("./TEV_fw.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

# files = glob.glob("./*_fw*")
files = ['./TEV-F.fa']

for f in files:
    # make a dictionary to collect the counts
    # fname = f[34:]
    fname = "TEV-wt-fw"
    dna_variant_counts[fname] = defaultdict(int)
    error_summary[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        number_of_nNs = var_record.seq.count('N') + var_record.seq.count('n')
        if number_of_nNs > 10:
            error_summary[fname]['nNs'] += 1
            continue
            
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            error_list.append(var_record)
        else:
            dna_variant_counts[fname][len(dna_d)] += 1
            if len(dna_d) > 1:
                printErrors(dna_d, var_record.seq, ref_seq.seq, True)
                
print(error_summary)
print(dna_variant_counts)

(('146', 's', 'A'), ('149', 's', 'C'))

0    5    10   15   20   25   30   35   40   45   50   55   60   65   70   75   80   85   90   95   
|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....
AAGAGCTTGTTTAAGGGGCCGCGTGATTACAACCCGATATCGAGCACCATTTGTCATTTGACGAATGAATCTGATGGGCACACAACATCGTTGTATGGTA
TTGGATTTGGTCCCTTCATCATTACAAACAAGCACTTGTTTAGAAAAACTAATGGAACACTGTTGGTCCAATCACTACATGGTGTATTCAAGGTCAAGAA
CACCACGACTTTGCAACAACACCTCATTGATGGGAGGGACATGATAATTATTCGCATGCCTAAGGATTTCCCACCATTTCCTCAAAAGCTGAAATTTAGA
GAGCCACAAAGGGAAGAGCGCATATGTCTTGTGACAACCAACTTCCAAACTAAGAGCATG
{'TEV-wt-fw': defaultdict(<class 'int'>, {'nNs': 1})}
{'TEV-wt-fw': defaultdict(<class 'int'>, {0: 1715, 1: 28, 2: 1})}


I think we can dismiss the read with 9 mutations, it appears to be a low quality read.

In [47]:
# sequencing error rate:
num_seq_errors = 28 + 2
num_seq_reads = (sum(dna_variant_counts['TEV-wt-fw'].values()))
print("errors per read:", num_seq_errors/num_seq_bases)
print("errors per base:", num_seq_errors/(num_seq_bases*len(ref_seq)))

errors per read: 0.017201834862385322
errors per base: 4.7782874617737003e-05


The expected ion torrent error rate is an error in 1.7% of reads, so we're about there.
https://bmcgenomics.biomedcentral.com/articles/10.1186/1471-2164-13-341

OPTION: 
It might be worth to look if there's a bias in random sequencing errors, there's probably lit for that.
Possibly keep checks on the reads for number of NNNs?

#### WT second half sequencing

In [50]:
# start with the forward sequenced on the WT
with open("./TEV_rv.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

# files = glob.glob("./*_fw*")
files = ['./TEV-R.fa']

for f in files:
    # make a dictionary to collect the counts
    # fname = f[34:]
    fname = "TEV-wt-rv"
    dna_variant_counts[fname] = defaultdict(int)
    error_summary[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        number_of_nNs = var_record.seq.count('N') + var_record.seq.count('n')
        if number_of_nNs > 10:
            error_summary[fname]['nNs'] += 1
            continue
            
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            error_list.append(var_record)
        else:
            dna_variant_counts[fname][len(dna_d)] += 1
            if len(dna_d) > 1:
                printErrors(dna_d, var_record.seq, ref_seq.seq, True)
                
print(error_summary)
print(dna_variant_counts)

(('275', 's', 'C'), ('277', 's', 'G'))

0    5    10   15   20   25   30   35   40   45   50   55   60   65   70   75   80   85   90   95   
|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....
TCTAGCATGGTGTCAGACACTAGTTGCACATTCCCTTCATCTGATGGCATATTCTGGAAGCATTGGATTCAAACCAAGGATGGGCAGTGTGGCAGTCCAT
TAGTATCAACTAGAGATGGGTTCATTGTTGGTATACACTCAGCATCGAATTTCACCAACACAAACAATTATTTCACAAGCGTGCCGAAAAACTTCATGGA
ATTGTTGACAAATCAGGAGGCGCAGCAGTGGGTTAGTGGTTGGCGATTAAATGCTGACTCAGTATTGTGGGGGGCCGATAAAGTTTTCATGGTGAAACCT
GAAGAGCCTTTTCAGCCAGTTAAGGAAGCGACTCAACTCATGAATTAA
(('119', 's', 'T'), ('259', 's', 'A'))

0    5    10   15   20   25   30   35   40   45   50   55   60   65   70   75   80   85   90   95   
|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....|....
TCTAGCATGGTGTCAGACACTAGTTGCACATTCCCTTCATCTGATGGCATATTCTGGAAGCATTGGATTCAAACCAAGGATGGGCAGTGTGGCAGTCCAT
TAGTATCAACTAGAGATGTGTTCATTGTTGGTATACACTCAGCATCGAATTTCACCAACACAA

In [51]:
# sequencing error rate second half:
num_seq_errors = 28 + 3
num_seq_reads = (sum(dna_variant_counts['TEV-wt-rv'].values()))
print("errors per read:", num_seq_errors/num_seq_bases)
print("errors per base:", num_seq_errors/(num_seq_bases*len(ref_seq)))

errors per read: 0.017775229357798166
errors per base: 5.107824528102921e-05


## Initial library composition

In [ ]:
files = glob.glob("./*_fw*")

In [ ]:
column_order = ['H_fw_1.fa', 'H_fw_3.fa', 'H_fw_5.fa', 
                'H_fw_7.fa', 'H_fw_9.fa', 'H_fw_11.fa', 
                'L_fw_1.fa','L_fw_3.fa', 'L_fw_5.fa', 
                'L_fw_7.fa', 'L_fw_9.fa', 'L_fw_11.fa']

In [ ]:
df = OrderedDict()

for fname in column_order:
    df[fname] = [dna_variant_counts[fname][i] for i in range(max_mutations +1)]
    
pd.DataFrame.from_dict(df)

In [ ]:
# now let's analyse to all second half sequences
with open("./TEV_rv.fa", "r") as f:
    wt_end =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

dna_variant_counts = {}

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_rv*")

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    dna_variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, wt_end.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var.record)
        else:
            dna_variant_counts[fname][len(dna_d)] += 1

observed_counts = [list(dna_variant_counts[i].keys()) for i in dna_variant_counts.keys()]
max_mutations = max(list(itertools.chain.from_iterable(observed_counts))) #  this tolerates an empty input file

In [ ]:
dna_variant_counts.keys()

In [ ]:
column_order = ['H_rv_1.fa', 'H_rv_3.fa','H_rv_5.fa',
                'H_rv_7.fa', 'H_rv_9.fa','H_rv_11.fa',
                'L_rv_1.fa', 'L_rv_3.fa', 'L_rv_5.fa',
                'L_rv_7.fa', 'L_rv_9.fa', 'L_rv_11.fa']

In [ ]:
df = OrderedDict()

for fname in column_order:
    df[fname] = [dna_variant_counts[fname][i] for i in range(max_mutations +1)]
    
pd.DataFrame.from_dict(df)

## Boxplots: number of nucleotide changes per variant

In [ ]:
def count_nt_subs_disaggregated(ref_file, files_to_analyse, dna_variant_counts):
    
    for f in files_to_analyse:
        # make a dictionary to collect the counts
        fname = f[34:]
        dna_variant_counts[fname] = []

        for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
            dna_d = find_dna_subs(var_record.seq, ref_file.seq, start_offset = 0, end_trail = 0)
            if dna_d is None:
                print(var.record)
            else:
                # filter out the indels
                # get the tuple[1] positions which should all be 's' for substitutions
                s = 0
                for p in dna_d:
                    if p[1] == 's':
                        s += 1
                    else:
                        print(dna_d)
                        break

                dna_variant_counts[fname].append(s)
                
    return dna_variant_counts

In [ ]:
dna_variant_counts = {}

with open("./TEV_rv.fa", "r") as f:
    wt_end =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_rv*")

dna_variant_counts = count_nt_subs_disaggregated(wt_end, files, dna_variant_counts)

# now add the forward files

with open("./TEV_fw.fa", "r") as f:
    wt_fw =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_fw*")

dna_variant_counts = count_nt_subs_disaggregated(wt_fw, files, dna_variant_counts)

In [ ]:
# set the desired order for boxplots
label_order = ['1_fw', '1_rv', '3_fw', '3_rv', '5_fw', '5_rv', '7_fw', '7_rv', '9_fw', '9_rv', '11_fw', '11_rv']

file_order_high = ['H_fw_1.fa', 'H_rv_1.fa',
                   'H_fw_3.fa', 'H_rv_3.fa',
                   'H_fw_5.fa', 'H_rv_5.fa', 
                   'H_fw_7.fa', 'H_rv_7.fa', 
                   'H_fw_9.fa', 'H_rv_9.fa',
                   'H_fw_11.fa', 'H_rv_11.fa']

file_order_low = ['L_fw_1.fa', 'L_rv_1.fa',  
                  'L_fw_3.fa', 'L_rv_3.fa', 
                  'L_fw_5.fa', 'L_rv_5.fa', 
                  'L_fw_7.fa', 'L_rv_7.fa', 
                  'L_fw_9.fa', 'L_rv_9.fa',
                  'L_fw_11.fa', 'L_rv_11.fa']             

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

In [ ]:
positions = [1, 2, 
             4, 5,
             7, 8,
             10, 11, 
             13, 14, 
             16, 17]

In [ ]:
plt.rcParams["figure.figsize"] = [7,7]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots()

ax.boxplot([dna_variant_counts[k] for k in file_order_high], positions = positions, widths = 0.75,
          showmeans=True, meanline=True)
ax.set_xticklabels(label_order)
ax.set_ylim([0, 28])

ax.tick_params(axis='x', labelrotation=45)
ax.set_xlabel('Sequencing population')
ax.set_ylabel('Number of nucleotide substitutions')
ax.set_title('High mutational load lineage (μ=1.5%)')

plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [7,7]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots()

ax.boxplot([dna_variant_counts[k] for k in file_order_low], positions = positions, widths = 0.75,
          showmeans=True, meanline=True)
ax.set_xticklabels(label_order)
ax.set_ylim([0, 28])

ax.tick_params(axis='x', labelrotation=45)
ax.set_xlabel('Sequencing population')
ax.set_ylabel('Number of nucleotide substitutions')
ax.set_title('Low mutational load lineage (μ=0.5%)')

plt.show()

# add both distributions together: model each sequence A added to each sequence B

In [ ]:
dna_variant_counts.keys()

In [ ]:
file_order_high

In [ ]:
round_order = ['Round 1', 'Round 3', 'Round 5', 'Round 7', 'Round 9', 'Round 11']

full_length_dna = {k : [] for k in round_order}

# let's start with high
f_order = file_order_high
# I can use the fact that the files are ordered in pairs for the boxplots
for i in range(len(round_order)):
    round_name = round_order[i]
    fw_name = f_order[2*i]
    rv_name = f_order[2*i+1]
    for variant_fw in dna_variant_counts[fw_name]:
        for variant_rv in dna_variant_counts[rv_name]:
            full_length_dna[round_name].append(variant_fw + variant_rv)

In [ ]:
plt.rcParams["figure.figsize"] = [7,7]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots()

ax.boxplot(full_length_dna.values(), showmeans=True, meanline=True, showfliers=False)
ax.set_xticklabels(round_order)
ax.set_ylim([0, 40])

ax.tick_params(axis='x', labelrotation=45)
ax.set_xlabel('Evolution population')
ax.set_ylabel('Number of nucleotide substitutions')
ax.set_title('High mutational load lineage (μ=1.5%)')

plt.show()

In [ ]:
for k, v in full_length_dna.items():
    try:
        m = statistics.median(v)
    except :
        continue
    print(k, "Median nt changes:", m)

In [ ]:
round_order = ['Round 1', 'Round 3', 'Round 5', 'Round 7', 'Round 9', 'Round 11']

full_length_dna = {k : [] for k in round_order}

# let's start with high
f_order = file_order_low
# I can use the fact that the files are ordered in pairs for the boxplots
for i in range(len(round_order)):
    round_name = round_order[i]
    fw_name = f_order[2*i]
    rv_name = f_order[2*i+1]
    print(rv_name, fw_name)
    for variant_fw in dna_variant_counts[fw_name]:
        for variant_rv in dna_variant_counts[rv_name]:
            full_length_dna[round_name].append(variant_fw + variant_rv)
            
plt.rcParams["figure.figsize"] = [7,7]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots()

ax.boxplot(full_length_dna.values(), showmeans=True, meanline=True, showfliers=False)
ax.set_xticklabels(round_order)
ax.set_ylim([0, 40])

ax.tick_params(axis='x', labelrotation=45)
ax.set_xlabel('Evolution population')
ax.set_ylabel('Number of nucleotide substitutions')
ax.set_title('Low mutational load lineage (μ=0.5%)')

plt.show()

In [ ]:
for k, v in full_length_dna.items():
    print(k, "Median nt changes:", statistics.median(v))

### Protein sequence variation

-	Forward files contain AA 2-121
-	Reverse files contain AA 122-236, also in forward read orientation; thus naming the files as reverse is misleading.


In [ ]:
def find_protein_diff(read, ref, verbose=False, start_offset=3, end_trail=3):

    # quality control
    if read is None:
        return None, None
    ends = findEnds(read, ref, start_offset)
#     if not endMatch(read, ref, ends):
#         return None, None

    newread = read
    newref = ref

    # scan reference triplet by triplet
    # move letters when encountering an indel
    prot_errors = []
    prot_short = []

    i = ends.get('aligned')
    ref_index = int((ends.get('aligned') - start_offset)/3) + 1  # reference amino acid index
    max_i = len(ref) - end_trail

    while i <= ends.get('end'):
        if i > max_i:
            break

        if newread is None:
            break
        ref_codon = newref[i:i+3]
        read_codon = newread[i:i+3]

        if '-' in read_codon:  # found a deletion
            # Check if this is the last acid, and it's incomplete, ignore it.
            if re.search('[ATGC]', str(newread[i + 3:])) is None:
                break

            if '-' in ref_codon:  # something very broken
                prot_errors.append((ref_index, 'f'))
                prot_short.append('f')
                break
            elif read_codon == '---':  # single codon deletion
                if ref_index > 0:
                    prot_errors += [(ref_index, 'd')]
                    prot_short.append(str(ref_index) + 'Δ')
                i += 3
                ref_index += 1

            else:  # check it's not a frame shift
                l = indel_len(newread, i)
                if l % 3 != 0:
                    prot_errors.append((ref_index, 'f'))
                    prot_short.append('f')
                    break
                # realign gap and repeat loop at same position to compare the codons
                gap = findGap(newread[i - 1:])
                gap = (gap[0] + i - 1, gap[1] + i - 1)
                newread = gapAlign(newread, gap, start_offset)
                continue

        elif '-' in ref_codon:  # found an insertion
            l = indel_len(newref, i)
            if l % 3 != 0:
                prot_errors.append((ref_index, 'f'))
                prot_short.append('f')
                break
            gap = findGap(newref[i-1:])
            if gap[0] == 1:  # insertion after codon
                insertion = newread[gap[0] + i - 1:gap[1] + i - 1]
                if '-' in insertion:
                    prot_errors.append((ref_index, 'f'))
                    prot_short.append('f')
                    break
                if ref_index > 0:
                    prot_errors.append((ref_index - 1, 'i', str(translate(insertion)))) # position before + insertion
                    stop, inslist = format_insertion(ref_index - 1, insertion)
                    prot_short += inslist
                    if stop:
                        break
                i += l
                ref_index += 1
            else:  # realign gap and repeat loop at same position to compare the codons
                gap = (gap[0] + i - 1, gap[1] + i - 1)
                newref = gapAlign(newref, gap, start_offset)
                continue

        elif translate(read_codon) != translate(ref_codon):  # must be a substitution
            if ref_index > 0:
                prot_errors.append((ref_index, 's', str(translate(read_codon))))
                prot_short.append(str(translate(ref_codon) + str(ref_index) + str(translate(read_codon))))
            if str(translate(read_codon)) == '*':
                break
            i += 3
            ref_index += 1

        else:
            i += 3
            ref_index += 1

    if verbose:
        print(prot_errors)

    if prot_short == []:
        short = 'wt'
    else:
        short = '/'.join(prot_short)

    return tuple(prot_errors), short


In [ ]:
# test protein_diff function for numbering

test_ref =  "AAGAGCTTGTTTAAGGGGCCGCGTGATTACAACCCGATATCGAGCACCATTTGTCATTTGACGAATGAATCTGATGGGCACACAACATCGTTGTATGGTATTGGATTTGGTCCCTTCATCATTACAAACAAGCACTTGTTTAGAAGAAATAATGGAACACTGTTGGTCCAATCACTACATGGTGTATTCAAGGTCAAGAACACCACGACTTTGCAACAACACCTCATTGATGGGAGGGACATGATAATTATTCGCATGCCTAAGGATTTCCCACCATTTCCTCAAAAGCTGAAATTTAGAGAGCCACAAAGGGAAGAGCGCATATGTCTTGTGACAACCAACTTCCAAACTAAGAGCATG"
test_read = "AAGAGCTTGTTTAAGGGGCCGCGTGATTACAACCCGATATCGAGCACCATTTGTCATTTGACGAATGAATCTGATGGGCACACAACATCGCTGTATGGTATTGGATTTGGTCCCTTCGTCATTACAAACAAGCACTTGTTTAGAAGAAATAATGGAACACTGTTGGTCCAATCACTACATGGTGTATTCAAGGTCAAGAACACCACGACTTTGCAACAACACCTCATTGATGGGAGGGACATGATAATTATTCGCATGCCTAAGGATTTCCCACCATTTCCTCAAAAGCTGACATTTAGAGAGCCACAAAGGGAAGAGCGCCTATGTCTTGTGACAACCAACTTCCAAACTAAGAGCAGG"

print(find_dna_subs(test_read, test_ref))
find_protein_diff(test_read, test_ref, start_offset = -3, end_trail = 0)

# since first AA should be position 2, use start_offset -3
# I need to disable end matching from the find_protein_diff function

In [ ]:
# now let's analyse all first half sequences
with open("./TEV_fw.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

variant_counts = {}

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_fw*")

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var.record)
        else:
            protein_d, prot_short = find_protein_diff(var_record.seq, ref_seq.seq, start_offset = -3, end_trail = 0)
            variant_counts[fname][len(protein_d)] += 1

In [ ]:
observed_counts = [list(variant_counts[i].keys()) for i in variant_counts.keys()]
max_mutations = max(list(itertools.chain.from_iterable(observed_counts))) #  this tolerates an empty input file
max_mutations

In [ ]:
column_order = ['H_fw_1.fa', 'H_fw_3.fa', 'H_fw_5.fa', 
                'H_fw_7.fa', 'H_fw_9.fa', 'H_fw_11.fa', 
                'L_fw_1.fa','L_fw_3.fa', 'L_fw_5.fa', 
                'L_fw_7.fa', 'L_fw_9.fa', 'L_fw_11.fa']

In [ ]:
df = OrderedDict()

for fname in column_order:
    df[fname] = [variant_counts[fname][i] for i in range(max_mutations +1)]
    
pd.DataFrame.from_dict(df)

### And repeat the process for the second half of protein sequences

In [ ]:
# now let's analyse all first half sequences
with open("./TEV_rv.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

variant_counts = {}

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_rv*")

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var.record)
        else:
            protein_d, prot_short = find_protein_diff(var_record.seq, ref_seq.seq, start_offset = -3, end_trail = 0)
            variant_counts[fname][len(protein_d)] += 1

In [ ]:
observed_counts = [list(variant_counts[i].keys()) for i in variant_counts.keys()]
max_mutations = max(list(itertools.chain.from_iterable(observed_counts))) #  this tolerates an empty input file
max_mutations

In [ ]:
column_order = ['H_rv_1.fa', 'H_rv_3.fa','H_rv_5.fa',
                'H_rv_7.fa', 'H_rv_9.fa','H_rv_11.fa',
                'L_rv_1.fa', 'L_rv_3.fa', 'L_rv_5.fa',
                'L_rv_7.fa', 'L_rv_9.fa', 'L_rv_11.fa']

In [ ]:
# make aggregate dict for easy variant counts presentation & mutation rate calculation
dct = OrderedDict()

for fname in column_order:
    dct[fname] = [variant_counts[fname][i] for i in range(max_mutations +1)]
    
df = pd.DataFrame.from_dict(dct)
df

In [ ]:
print(ref_seq)

## Expand the boxplots for counting mutations to consider different types of AA mutations

In [ ]:
# start looking at the type of protein change: synonimous, non-synonimous, stop
with open("./TEV_fw.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

variant_counts = {}

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_fw*")

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var.record)
        else:
            protein_d, prot_short = find_protein_diff(var_record.seq, ref_seq.seq, start_offset = -3, end_trail = 0)
